In [1]:
! pip install -q kaggle
from google.colab import files

files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                  title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
---------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
sudarshan24byte/online-food-dataset                  Online Food Dataset                             3KB  2024-03-02 18:50:30          17476        346  0.9411765        
syedanwarafridi/vehicle-sales-data                   Vehicle Sales Data                             19MB  2024-02-21 20:16:17          23958        386  1.0              
alistairking/natural-gas-usage                       Natural Gas Usage (2014-Now)                  490KB  2024-03-31 02:31:28            556         28  1.0              
bhavikjikadara/student-study-performance             Student Study Performance                       9KB  2024-

In [2]:
!kaggle datasets download -d gpiosenka/car-parts-40-classes

 99% 298M/300M [00:15<00:00, 26.8MB/s]
100% 300M/300M [00:15<00:00, 20.7MB/s]


In [3]:
# Extracting the compressed dataset.
from zipfile import ZipFile
data_path = '/content/car-parts-40-classes.zip'
with 	ZipFile(data_path, 'r') as zip : zip.extractall()

In [4]:
import pandas as pd
import numpy as np
from numpy import random
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import albumentations as A
import time
from tqdm import tqdm
from sklearn.metrics import f1_score
from IPython.display import YouTubeVideo
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [5]:
limiter = int(input('Entrez une valeur pour le nombre maximum d\'images par classe : '))
data_path = r'car parts 50'
trainpath = os.path.join(data_path, 'train')
validpath = os.path.join(data_path, 'valid')
testpath = os.path.join(data_path, 'test')
paths = [trainpath, validpath, testpath]
names = ['train', 'valid', 'test']

for i in range(3):
    path = paths[i]
    name = names[i]
    classes = os.listdir(path)
    filepaths = []
    labels = []

    for klass in classes:
        classpath = os.path.join(path, klass)
        flist = os.listdir(classpath)

        if len(flist) > limiter:
            flist = np.random.choice(flist, limiter, replace=False)

        for f in flist:
            fpath = os.path.join(classpath, f)
            filepaths.append(fpath)
            labels.append(klass)

    Fseries = pd.Series(filepaths, name='filepaths')
    Lseries = pd.Series(labels, name='labels')

    if name == 'train':
        train_df = pd.concat([Fseries, Lseries], axis=1)
    elif name == 'valid':
        valid_df = pd.concat([Fseries, Lseries], axis=1)
    else:
        test_df = pd.concat([Fseries, Lseries], axis=1)

num_of_classes = len(classes)
print(f'test_df length= {len(test_df)}  train_df lngth= {len(train_df)}  valid_df length= {len(valid_df)}')
train_balance = train_df['labels'].value_counts()
print(f'the train_set has {num_of_classes} classes, class balance is:\n {train_balance}')

Entrez une valeur pour le nombre maximum d'images par classe : 255
test_df length= 250  train_df lngth= 8739  valid_df length= 250
the train_set has 50 classes, class balance is:
 IGNITION COIL           200
IDLER ARM               198
STARTER                 195
CARBERATOR              192
ALTERNATOR              190
RADIO                   189
OVERFLOW TANK           189
SHIFT KNOB              189
ENGINE BLOCK            188
FUEL INJECTOR           188
BRAKE PAD               188
OIL PAN                 187
OIL PRESSURE SENSOR     187
TORQUE CONVERTER        187
SIDE MIRROR             186
GAS CAP                 186
SPOILER                 185
AIR COMPRESSOR          185
CYLINDER HEAD           184
OXYGEN SENSOR           182
PRESSURE PLATE          182
THERMOSTAT              182
CLUTCH PLATE            181
CRANKSHAFT              180
INSTRUMENT CLUSTER      180
RIM                     179
WINDOW REGULATOR        179
VACUUM BRAKE BOOSTER    178
BATTERY                 177
BRAKE RO

In [6]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Charger le modèle VGG16 pré-entraîné
model = VGG16(weights='imagenet', include_top=False)

# Chemin du dossier contenant les images augmentées
chemin_dossier = '/content/car parts 50/test'

# Dictionnaire pour stocker les vecteurs de caractéristiques de toutes les classes
vecteurs_caracteristiques_par_classe = {}

# Liste des classes (nom des sous-dossiers)
classes = os.listdir(chemin_dossier)

# Parcourir chaque classe (sous-dossier)
for classe in classes:
    chemin_classe = os.path.join(chemin_dossier, classe)
    # Vérifier si c'est un dossier
    if os.path.isdir(chemin_classe):
        # Liste pour stocker les vecteurs de caractéristiques de cette classe
        vecteurs_caracteristiques_classe = []
        # Liste des fichiers dans le dossier
        fichiers = os.listdir(chemin_classe)
        # Parcourir chaque image dans le dossier
        for fichier in fichiers:
            # Construire le chemin complet de l'image
            chemin_image = os.path.join(chemin_classe, fichier)
            # Charger et prétraiter l'image
            img = image.load_img(chemin_image, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            # Passer l'image à travers le modèle VGG16
            features = model.predict(x)
            # Aplatir les activations pour obtenir un vecteur de caractéristiques
            features_vector = features.flatten()
            # Ajouter le vecteur de caractéristiques à la liste de cette classe
            vecteurs_caracteristiques_classe.append(features_vector)
        # Ajouter la liste de vecteurs de caractéristiques de cette classe au dictionnaire
        vecteurs_caracteristiques_par_classe[classe] = vecteurs_caracteristiques_classe

# Sauvegarder les vecteurs de caractéristiques dans un fichier NumPy
np.savez('vecteurs_caracteristiques_test.npz', **vecteurs_caracteristiques_par_classe)

1/1 [==============================] - 0s 464ms/step
